In [1]:
from pytube import YouTube
from pydub import AudioSegment
from pytube import Playlist
import os
import re
from glob import glob
from tqdm import tqdm
import pandas as pd
import random

/tmp/ipykernel_4021032/1090648631.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def flow():
    """
    - input the playlists for each qare2

    - logg in the qare2 and the sura (`logg_in()`)

    - check , if the qare2 (extract the name from the title using `get_reciter()`)\
      has more than 4 occurences ( using the function `get_thresh()`) for the same sura then find another sura\
      for the same qare2 using the function `random_slicer()` which slices three random links from the urls in the\
      playlist.
    
    - download the sura (keeping the name of the reciter and the sura)

    - split the sura into 100 segments (1 second each)

    - save them with the prefix containing the qare2's name and index
  

    """
pass


In [3]:
RECITERS = [
    # "احمد الشلبي",
    # "علاء عقل",
    # "اسماعيل القاضي",
    # "علاء ياسر",
    "محمد حجازي",
    "محمد نصر الدين",
    # "مختار الحاج",
    # "إرويانتو",
    # "زين ابو كوثر",
    # "Zain Abu Kautsar",
    "حسين عبد الظاهر",
    "طارق محمد",
    "عبد الباسط",
    "المنشاوي",
    "عبد العزيز سحيم",
]


In [4]:
df = pd.DataFrame(columns = ['reciter','sura'])

In [5]:
def get_thresh(df,reciter,sura):
    # Filter DataFrame based on the conditions
    filtered_df = df[df['reciter'] == reciter]
    if len(filtered_df[filtered_df['sura'] == sura]) > 4:
         return True

In [6]:
def get_reciter(RECITERS, string):
    matches = [substring for substring in RECITERS if substring.lower() in string.lower()]
    
    if not matches:
        # No match found
        return None  # or raise an exception, return a default value, etc.
    
    return matches[0]

def get_sura(title):
    match = re.search(r"سورة\s[الء-ي]+", title)
    
    if match is None:
        # No match found
        return None  # or raise an exception, return a default value, etc.
    
    return match.group(0)

In [7]:
def logging_in(reciter ,title,df):
    """
    logginf in the name of the reciter and the name of the sura
    """
    sura = get_sura(title)
    reciter_name = reciter
    new_row = {'reciter': reciter_name, 'sura':sura}
    print(f"logging in {reciter} for {sura}")
    df.loc[len(df)] = new_row
    return df

In [8]:
def random_slicer(input_list, num_items):
    """
    slices three random items from a list
    """
    # Make a copy of the input list to avoid modifying the original list
    input_list = list(input_list)
    shuffled_list = input_list.copy()

    # Shuffle the list randomly
    random.shuffle(shuffled_list)

    # Slice the first 'num_items' elements from the shuffled list
    sliced_items = shuffled_list[:num_items]

    return sliced_items

In [9]:
def get_title(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only("mp4")
    return audio_stream.title

In [10]:
def download_youtube_audio(url, output_path):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True)
    audio_stream[0].download(output_path)

In [11]:
def download_audio_from_playlist(url, output_path, df):
    try:
        playlist = Playlist(url)
        urls = playlist.video_urls

        sura = None  # Initialize sura outside the loop

        while True:
            shuffled_playlist = random_slicer(urls, 10)

            for url in shuffled_playlist:
                title = get_title(url)
                reciter = get_reciter(RECITERS, title)

                if reciter is None:
                    reciter = get_reciter(RECITERS, playlist.title)
                else:
                    sura = get_sura(title)
                    df_new = logging_in(reciter, title, df)

                    if sura is not None:
                        if get_thresh(df_new, sura, reciter):
                            print('Reslicing the playlist')
                            break  # Break out of the inner loop to re-randomize the playlist
                        else:
                            try:
                                download_youtube_audio(url, output_path)
                                print(f"Downloading {sura} for {reciter}\n Title of the video is {title}")
                            except Exception as e:
                                print(f"Error processing video: {title} - {e}")
    
    except:
        print("Couldn't return playlist")
        return title, reciter, sura, df_new


In [12]:
def split_audio(input_path, prefix, output_path, segment_length=1000, export_format="wav", bitrate="192k"):
    try:
        audio = AudioSegment.from_file(input_path)

        # Calculate the number of segments
        num_segments = len(audio) // segment_length

        # Split the audio into segments
        for i in range(num_segments):
            start_time = i * segment_length
            end_time = (i + 1) * segment_length
            segment = audio[start_time:end_time]

            # Save each segment
            output_filename = f"{prefix}_segment_{i + 1}.{export_format}"
            segment.export(os.path.join(output_path, output_filename), format=export_format, bitrate=bitrate)
    except:
        print(f"couldn't save {output_filename}")


In [13]:
RECITERS

['محمد حجازي',
 'محمد نصر الدين',
 'حسين عبد الظاهر',
 'طارق محمد',
 'عبد الباسط',
 'المنشاوي',
 'عبد العزيز سحيم']

In [14]:
# List of YouTube video URLs
video_urls = {
        #  "عبد الباسط":[
        #                "https://www.youtube.com/playlist?list=PLdgJ3Z9AFIkVxK28C8vDmeMXDHvpCTwGi",
        #                "https://www.youtube.com/playlist?list=PL7FQ8_TtkWWGPqGeX1t9Y7Ls4c_n6Fx8_"],

        #  "المنشاوي":["https://www.youtube.com/playlist?list=PLr2rWP0a1jdHTsYPbz0hiDfa0-VeRFeJ2",
        #              "https://www.youtube.com/playlist?list=PLMrC4AI4wdRrSQLGtK4nmSJXlC1sZLhy8",
        #              "https://www.youtube.com/playlist?list=PLr2rWP0a1jdHQWhdLZDW5Gsri4_cPqD8_"],

       "طارق محمد":[

                    "https://www.youtube.com/playlist?list=PLZ8mb5KlWBIyEfWHl2cvH-817c630_dcr"]
        

}

for name , playlists in video_urls.items():
    os.makedirs(f"data/{name}", exist_ok=True)
    print(f'creating directory for {name}')
    for playlist in playlists:
             
        relative_output_dir_path = os.path.relpath(f"data/{name}")

        title,reciter,sura,_ = download_audio_from_playlist(playlist, relative_output_dir_path, df)

        videos_for_conv_paths = glob(os.path.join(relative_output_dir_path,"*.mp4"))
        for i,video in tqdm(enumerate(videos_for_conv_paths)):

            prefix = f"{title[:-1]}_{reciter}_{sura}_{i + 1}"

            split_audio(video,prefix, relative_output_dir_path)
            
        for file in videos_for_conv_paths:
            # if filename.endswith(".mp4"):
            #     file_path = os.path.join(relative_output_dir_path, filename)
                os.remove(file)
                print(f"Deleted: {file}")
        print("Restarting")
        

creating directory for طارق محمد


logging in طارق محمد for سورة طه
 Title of the video is القارئ طارق محمد ماتيسرمن سورة طه ﴿إِنَّمَاۤ إِلَـٰهُكُمُ ٱللَّهُ ٱلَّذِی لَاۤ إِلَـٰهَ إِلَّا هُوَ﴾
logging in طارق محمد for None
logging in طارق محمد for None
logging in طارق محمد for سورة يس
 Title of the video is من سورة يس Surat YaSin | تلاوة هادئة القارئ طارق محمد - قرآن شاشة سوداء | كرومات قران جاهزه للتصميم🍂
logging in طارق محمد for None
logging in طارق محمد for None
logging in طارق محمد for سورة يس
 Title of the video is القاريء طارق محمد - تلاوة ندية عذبة رقراقة من سورة يس 🎧💜🌴| .
logging in طارق محمد for سورة طه
 Title of the video is ما تيسر من سورة طه | القارئ طارق محمد || Surah Taahaa قرآن كريم || اجمل حالات واتس قرآنية🌿🌹
logging in طارق محمد for سورة الفرقان
 Title of the video is ويوم يعض الظالم على يديه - سورة الفرقان - القارئ طارق محمد || القرآن الكريم شاشة سوداء 🌱
logging in طارق محمد for None
logging in طارق محمد for سورة الكهف
 Title of the video is القارىء طارق محمد - سورة الكهف | مقاطع قرآن قصيرة وحالات واتس 

21it [00:15,  1.34it/s]

Deleted: data/طارق محمد/القارئ طارق محمد - سورة الفرقان  مقاطع قرآن قصيرة وحالات واتس إسلامية 🌱.mp4
Deleted: data/طارق محمد/القارئ طارق محمد - سورة النازعات  مقاطع قرآن قصيرة وحالات واتس إسلامية 🌱.mp4
Deleted: data/طارق محمد/القارئ طارق محمد - سورة ق  مقاطع قرآن قصيرة وحالات واتس إسلامية 🌱.mp4
Deleted: data/طارق محمد/القارئ طارق محمد ماتيسرمن سورة طه ﴿إِنَّمَاۤ إِلَـٰهُكُمُ ٱللَّهُ ٱلَّذِی لَاۤ إِلَـٰهَ إِلَّا هُوَ﴾.mp4
Deleted: data/طارق محمد/القارئ طارق محمد من سورة ال عمران 🍃🌷.mp4
Deleted: data/طارق محمد/القارىء طارق محمد - إن الله لا يغير ما بقوم - سورة الرعد  مقاطع قرآن قصيرة وحالات واتس إسلامية 🌱.mp4
Deleted: data/طارق محمد/القارىء طارق محمد - سورة الفرقان  مقاطع قرآن قصيرة وحالات واتس إسلامية 🌱.mp4
Deleted: data/طارق محمد/القارىء طارق محمد - سورة الكهف  مقاطع قرآن قصيرة وحالات واتس إسلامية 🌱.mp4
Deleted: data/طارق محمد/القاريء طارق محمد - تلاوة ندية عذبة رقراقة من سورة يس 🎧💜🌴 .mp4
Deleted: data/طارق محمد/تلاوة جميلة من سورة آل عمران القارئ طارق محمد  قرآن شاشة سوداء كرومات قران 

In [15]:
df

,reciter,sura
0,طارق محمد,سورة طه
1,طارق محمد,None
2,طارق محمد,None
3,طارق محمد,سورة يس
4,طارق محمد,None
...,...,...
77,طارق محمد,None
78,طارق محمد,سورة النازعات
79,طارق محمد,سورة طه
80,طارق محمد,سورة الرعد
